# How Do I: Use Form-Filling in Go-Bot Skill with RASA DSLs (v1)

Form-filling is the task of collection some slots information from user.

## Form Filling In DeepPavlov Go-Bot Framework
Form filling takes parts in two config files: `domain` and `stories` configs. 

### `Domain.yml` configuration for formfilling
One should define forms in the `forms` section of `domain.yml`.
Each form is an object listing the slots that have to be filled for form to succeed. See `zoo_form` as a reference. `from_entity` is the only supported format for now.

```
forms:
  zoo_form:
    animal:
      - type: from_entity
        entity: animal
```

### `Stories.md` configuration for formfilling

form filling and the action that should happen when the ffiling is done are listed as two consecutive turns in `stories.md` as it is done in RASA 1.X DSL. See e.g. formfilling story below:

```
## formfilling dialogue
* greet
  - form{"name": "zoo_form"}
  - utter_api_call
```

##Minimal demo
Minimal demo 

In [1]:
DP_MIN_DEMO_DIR = "dp_minimal_demo_dir"  # we will work in this folder
import os

%cd /content
os.makedirs(DP_MIN_DEMO_DIR, exist_ok=True)

/content


### `Stories.md`

In [2]:
STORIES_PATH = f"{DP_MIN_DEMO_DIR}/stories.md"

In [3]:
%%writefile {STORIES_PATH}

## greet dialogue
* greet
  - utter_greet

## formfilling dialogue
* greet
  - form{"name": "zoo_form"}
  - utter_api_call


Writing dp_minimal_demo_dir/stories.md


In [4]:
!cp {DP_MIN_DEMO_DIR}/stories.md {DP_MIN_DEMO_DIR}/stories-trn.md
!cp {DP_MIN_DEMO_DIR}/stories.md {DP_MIN_DEMO_DIR}/stories-tst.md 
!cp {DP_MIN_DEMO_DIR}/stories.md {DP_MIN_DEMO_DIR}/stories-val.md

### `NLU.md`

In [5]:
NLU_PATH=f"{DP_MIN_DEMO_DIR}/nlu.md"

In [6]:
%%writefile {NLU_PATH}
## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks

## intent:inform_animal
- I'm looking for a [cat](animal)
- I am looking for a [cat](animal)
- I am looking for a [dog](animal)
- I am looking for a [fox](animal)

## intent:inform_time
- [Evening](time) is fine
- I think that this [morning](time)
- I am looking for a cat this [morning](time)

Writing dp_minimal_demo_dir/nlu.md


###`Domain.yml`

In [7]:
DOMAIN_PATH=f"{DP_MIN_DEMO_DIR}/domain.yml"

In [8]:
%%writefile {DOMAIN_PATH}
intents:
  - greet
  - inform_animal
  - inform_time

actions:
  - utter_api_call

slots:
  time:
    type: str
  animal:
    type: str

responses:
  utter_greet:
    - text: Hi
  utter_ask_animal:
    - text: Could you please tell me which animal ure lookin for
  utter_ask_time:
    - text: What's the desired time
  
forms:
    zoo_form:
        animal:
          - type: from_entity
            entity: animal
        time:
          - type: from_entity
            entity: time

Writing dp_minimal_demo_dir/domain.yml


### Install the deeppavlov library

In [9]:
!pip install git+https://github.com/deepmipt/DeepPavlov.git@feature/gobot_naive_formfilling

  Cloning https://github.com/deepmipt/DeepPavlov.git (to revision feature/gobot_naive_formfilling) to /tmp/pip-req-build-bdygyo_5
  Running command git clone -q https://github.com/deepmipt/DeepPavlov.git /tmp/pip-req-build-bdygyo_5
  Running command git checkout -b feature/gobot_naive_formfilling --track origin/feature/gobot_naive_formfilling
  Switched to a new branch 'feature/gobot_naive_formfilling'
  Branch 'feature/gobot_naive_formfilling' set up to track remote branch 'feature/gobot_naive_formfilling' from 'origin'.
     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 1.5MB 34.3MB/s 
     |████████████████████████████████| 20.1MB 1.2MB/s 
     |████████████████████████████████| 10.4MB 35.0MB/s 
     |████████████████████████████████| 512kB 42.3MB/s 
     |████████████████████████████████| 7.3MB 27.7MB/s 
     |██████████████████████

### Imports

In [10]:
import json
import os

from deeppavlov import build_model, train_model
from deeppavlov.core.common.file import read_json

### Write the `slotfiller` component chainer config

In [11]:
SLOTFILL_CONFIG_PATH = f"{DP_MIN_DEMO_DIR}/slotfill_config.json"

In [12]:
%%writefile {SLOTFILL_CONFIG_PATH}
{
  "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "pipe": [
      {
        "in": ["x"],
        "class_name": "lazy_tokenizer",
        "out": ["x_tokens"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "str_lower",
        "out": ["x_lower"]
      },
      {
        "in": ["x_lower"],
        "class_name": "slotfill_raw_rasa",
        "save_path": "{DATA_PATH}",
        "load_path": "{DATA_PATH}",
        "out": ["slots"]
      }
    ],
    "out": ["slots"]
  },
  "metadata": {
    "variables": {
      "DATA_PATH": "dp_minimal_demo_dir"
    }
  }
}

Writing dp_minimal_demo_dir/slotfill_config.json


In [13]:
slotfill_config = read_json(SLOTFILL_CONFIG_PATH)
slotfiller = build_model(slotfill_config)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


In [14]:
slotfiller(["Hello i'm looking for a cat"])

[{'animal': 'cat'}]

### Train the Go-Bot

In [15]:
GOBOT_CONFIG_PATH = f"{DP_MIN_DEMO_DIR}/gobot_config.json"

In [16]:
%%writefile {GOBOT_CONFIG_PATH}
{
  "dataset_reader": {
    "class_name": "md_yaml_dialogs_reader",
    "data_path": "{DATA_PATH}",
    "ignore_slots": false
  },
  "dataset_iterator": {
    "class_name": "dialog_iterator"
  },
  "chainer": {
    "in": [
      "x"
    ],
    "in_y": [
      "y"
    ],
    "out": [
      "y_predicted"
    ],
    "pipe": [
      {
        "class_name": "deeppavlov.models.go_bot.wrapper:DialogComponentWrapper",
        "component": {
          "class_name": "split_tokenizer"
        },
        "in": [
          "x"
        ],
        "out": [
          "x_tokens"
        ]
      },
      {
        "id": "word_vocab",
        "class_name": "simple_vocab",
        "fit_on": [
          "x_tokens"
        ],
        "save_path": "{MODEL_PATH}/word.dict",
        "load_path": "{MODEL_PATH}/word.dict"
      },
      {
        "class_name": "go_bot",
        "load_path": "{MODEL_PATH}/model",
        "save_path": "{MODEL_PATH}/model",
        "in": [
          "x"
        ],
        "in_y": [
          "y"
        ],
        "out": [
          "y_predicted"
        ],
        "main": true,
        "debug": false,
        "learning_rate": 0.003,
        "learning_rate_drop_patience": 5,
        "learning_rate_drop_div": 10,
        "momentum": 0.95,
        "optimizer": "tensorflow.train:AdamOptimizer",
        "clip_norm": 2,
        "dropout_rate": 0.4,
        "l2_reg_coef": 0.0003,
        "hidden_size": 128,
        "dense_size": 160,
        "word_vocab": "#word_vocab",
        "database": null,
        "nlg_manager": {
          "class_name": "gobot_json_nlg_manager",
          "data_path": "{DATA_PATH}",
          "dataset_reader_class": "md_yaml_dialogs_reader",
          "actions2slots_path": "{DATA_PATH}/dstc2-actions2slots.json",
          "api_call_action": null
        },
        "api_call_action": null,
        "use_action_mask": false,
        "slot_filler": {
          "config_path": "dp_minimal_demo_dir/slotfill_config.json"
        },
        "intent_classifier": null,
        "embedder": null,
        "bow_embedder": null,
        "tokenizer": {
          "class_name": "split_tokenizer"
        },
        "tracker": {
          "class_name": "featurized_tracker",
          "slot_names": [
            "animal", "time"
          ]
        }
      }
    ]
  },
  "train": {
    "epochs": 200,
    "batch_size": 4,
    "metrics": [
      "per_item_action_accuracy"
    ],
    "validation_patience": 10,
    "val_every_n_batches": 15,
    "log_every_n_batches": 15,
    "log_on_k_batches": -1,
    "show_examples": false,
    "evaluation_targets": [
      "valid",
      "test"
    ],
    "class_name": "nn_trainer"
  },
  "metadata": {
    "variables": {
      "ROOT_PATH": "~/.deeppavlov",
      "CONFIGS_PATH": "{DEEPPAVLOV_PATH}/configs",
      "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
      "DATA_PATH": "dp_minimal_demo_dir",
      "MODELS_PATH": "{ROOT_PATH}/models",
      "MODEL_PATH": "dp_minimal_demo_dir"
    },
    "requirements": [
      "{DEEPPAVLOV_PATH}/requirements/tf.txt",
      "{DEEPPAVLOV_PATH}/requirements/gensim.txt",
      "{DEEPPAVLOV_PATH}/requirements/spacy.txt",
      "{DEEPPAVLOV_PATH}/requirements/en_core_web_sm.txt"
    ],
    "download": []
  }
}

Writing dp_minimal_demo_dir/gobot_config.json


In [17]:
!python -m deeppavlov install {GOBOT_CONFIG_PATH}

     |████████████████████████████████| 24.2MB 5.9MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 110.5MB 80kB/s 
     |████████████████████████████████| 512kB 36.6MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 3.8MB 28.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=df64dbcaf33b275d067ca0ed9d534fa0e806076dc7a1ec5314b65e337463989c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uni

In [18]:
gobot_config = read_json(GOBOT_CONFIG_PATH)

gobot_config["chainer"]["pipe"][-1]["tracker"]["domain_yml_path"] = DOMAIN_PATH
gobot_config["chainer"]["pipe"][-1]["tracker"]["stories_yml_path"] = STORIES_PATH
gobot_config['chainer']['pipe'][-1]['slot_filler'] = {'config_path': SLOTFILL_CONFIG_PATH}

json.dump(gobot_config, open('gobot_config.json', 'wt'))

bot = train_model(gobot_config, download=False)

2020-12-21 09:08:12.690 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpv8sued_o]
2020-12-21 09:08:12.696 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpuqnpqbo6]
2020-12-21 09:08:12.701 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpkb9v7x1e]
2020-12-21 09:08:12.716 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/dp_minimal_demo_dir/word.dict]


2020-12-21 09:08:14.836 INFO in 'deeppavlov.models.go_bot.nlg.nlg_manager'['mock_json_nlg_manager'] at line 94: INSIDE MockJSONNLGManager _load_actions2slots_mapping(): actions2slots_json_path=/content/dp_minimal_demo_dir/dstc2-actions2slots.json DOES NOT EXIST. initialized actions2slots mapping with an empty one: {}
2020-12-21 09:08:14.849 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpaet6g6sn]
2020-12-21 09:08:14.861 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpgkp2m65m]
2020-12-21 09:08:14.876 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp52shz7ii]


zoo_form
zoo_form




Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-12-21 09:08:15.779 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-12-21 09:08:16.5 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best per_item_action_accuracy of 0.4


{"valid": {"eval_examples_count": 24, "metrics": {"per_item_action_accuracy": 0.4}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


2020-12-21 09:08:24.256 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-12-21 09:08:24.259 INFO in 'deeppavlov.models.go_bot.nlg.nlg_manager'['mock_json_nlg_manager'] at line 94: INSIDE MockJSONNLGManager _load_actions2slots_mapping(): actions2slots_json_path=/content/dp_minimal_demo_dir/dstc2-actions2slots.json DOES NOT EXIST. initialized actions2slots mapping with an empty one: {}
2020-12-21 09:08:24.270 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpb7o0j9sp]
2020-12-21 09:08:24.284 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpswq7zy_3]
2020-12-21 09:08:24.296 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpmf001tva]


{"valid": {"eval_examples_count": 24, "metrics": {"per_item_action_accuracy": 0.875}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 24, "metrics": {"per_item_action_accuracy": 0.875}, "time_spent": "0:00:01"}}
zoo_form
zoo_form


2020-12-21 09:08:25.117 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-12-21 09:08:25.124 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_minimal_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


### interact with the bot

Let's interact with the bot to see if it performs formfilling properly

In [19]:
replices = [
    "start",
    "hi bot! i'd like to go to the zoo",
    "i'd love to see a wild cat",
    "evenin would be great!"
]
bot.reset()
for rep in replices:
    res_bot = bot([rep])[0][0].to_serializable_dict()
    print(str(res_bot))

zoo_form
{'start': {}}
{'utter_ask_animal': {}}
{'utter_ask_time': {}}
{'utter_api_call': {}}
